In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import cv2
import os
import numpy as np
import pandas as pd

In [31]:
# dimensions of our images.
img_width, img_height = 150, 150

In [40]:
# Setting up few parameters
train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [41]:
# Defining image inpur shape
#channels_first is for theano and channels_last for tensorflow
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [42]:
## SEQUENTIAL MODEL ##

#Layer 1
model = Sequential()
model.add(Conv2D(48, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Layer 2
model = Sequential()
model.add(Conv2D(48, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
          
# Flatten and Sigmoid Layer with dropout
model.add(Flatten())
model.add(Dense(72))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [43]:
# Compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
# We need to augment our data to capture all aspects of image since we're just training on 2K images
train_aug = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [45]:
# For test data, we are just using rescaling
test_aug = ImageDataGenerator(rescale=1. / 255)

In [46]:
train_generator = train_aug.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.


In [47]:
validation_generator = test_aug.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 800 images belonging to 2 classes.


In [48]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)
# WoW !! We have loss: 0.2200 - accuracy: 0.9085. 
# Remember, we just used 2k images

Epoch 1/50
125/125 [==============================] - 39s 310ms/step - loss: 0.9979 - accuracy: 0.5010 - val_loss: 0.6918 - val_accuracy: 0.5337
Epoch 2/50
125/125 [==============================] - 39s 312ms/step - loss: 0.6938 - accuracy: 0.5060 - val_loss: 0.6885 - val_accuracy: 0.6263
Epoch 3/50
125/125 [==============================] - 39s 316ms/step - loss: 0.6908 - accuracy: 0.5500 - val_loss: 0.6769 - val_accuracy: 0.6313
Epoch 4/50
125/125 [==============================] - 39s 315ms/step - loss: 0.6769 - accuracy: 0.5440 - val_loss: 0.6673 - val_accuracy: 0.5663
Epoch 5/50
125/125 [==============================] - 39s 313ms/step - loss: 0.6609 - accuracy: 0.5935 - val_loss: 0.6366 - val_accuracy: 0.6212
Epoch 6/50
125/125 [==============================] - 40s 317ms/step - loss: 0.6570 - accuracy: 0.6365 - val_loss: 0.6282 - val_accuracy: 0.6538
Epoch 7/50
125/125 [==============================] - 40s 320ms/step - loss: 0.6251 - accuracy: 0.6475 - val_loss: 0.6057 - val_ac